In [289]:
import pandas as pd
import gzip
import simplejson as json


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [437]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\brian\.conda\envs\metis\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df = getDF(r'C:\Users\brian\NLP\data\Electronics_5.json.gz')

In [113]:
import numpy as np

## Data Prep

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6739590 entries, 0 to 6739589
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   overall         float64
 1   vote            object 
 2   verified        bool   
 3   reviewTime      object 
 4   reviewerID      object 
 5   asin            object 
 6   style           object 
 7   reviewerName    object 
 8   reviewText      object 
 9   summary         object 
 10  unixReviewTime  int64  
 11  image           object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 623.5+ MB


In [3]:
cell_df = getDF(r'C:\Users\brian\NLP\data\Cell_Phones_and_Accessories_5.json.gz')

In [35]:
meta_cell = getDF(r'C:\Users\brian\NLP\data\meta_Cell_Phones_and_Accessories.json.gz')

In [4]:
meta_elec = getDF(r'C:\Users\brian\NLP\data\meta_Electronics.json.gz')

In [5]:
meta_elec.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 786445 entries, 0 to 786444
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   category         786445 non-null  object
 1   tech1            786445 non-null  object
 2   description      786445 non-null  object
 3   fit              786445 non-null  object
 4   title            786445 non-null  object
 5   also_buy         786445 non-null  object
 6   tech2            786445 non-null  object
 7   brand            786445 non-null  object
 8   feature          786445 non-null  object
 9   rank             786445 non-null  object
 10  also_view        786445 non-null  object
 11  main_cat         786445 non-null  object
 12  similar_item     786445 non-null  object
 13  date             786445 non-null  object
 14  price            786445 non-null  object
 15  asin             786445 non-null  object
 16  imageURL         786445 non-null  object
 17  imageURLHi

In [28]:
meta_elec.iloc[105000]

category           [Electronics, Camera & Photo, Bags & Cases, Ca...
tech1                                                               
description        [Case Logic DCB-36 Compact Camcorder Case (BRO...
fit                                                                 
title                          DCB-36 Compact Camcorder Case (BROWN)
also_buy                                                          []
tech2                                                               
brand                                                     Case Logic
feature            [Fits most digital camcorders, Never miss a sh...
rank               [>#1,301 in Camera & Photo > Camera & Photo Ac...
also_view                                                         []
main_cat                                              Camera & Photo
similar_item                                                        
date                                                     May 5, 2007
price                             

In [7]:
# perform a left join onto the review dataframe so that we can know what product each datapoint represents
full_elec = pd.merge(df, meta_elec, how='left', on=['asin', 'asin'])

In [36]:
full_cell = pd.merge(cell_df, meta_cell, how='left', on=['asin', 'asin'])

In [63]:
# need to find all televisions
# extract all data points with television in 'title' column
tv_df = full_elec[full_elec['title'].astype(str).str.contains("TV", na='False')]

In [68]:
tv_df.iloc[5000]

overall                                                          5.0
vote                                                             NaN
verified                                                        True
reviewTime                                                04 7, 2015
reviewerID                                            A30FR2NT1PD45Q
asin                                                      B00068YUN4
style                                                            NaN
reviewerName                                                  Glynis
reviewText         Nice quality product and would recommend this ...
summary                                                   Five Stars
unixReviewTime                                            1428364800
image                                                            NaN
category           [Electronics, Accessories & Supplies, Audio & ...
tech1                                                               
description        [The DS-2000 un

In [67]:
tv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 376892 entries, 173 to 7063268
Data columns (total 30 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   overall          376892 non-null  float64
 1   vote             62981 non-null   object 
 2   verified         376892 non-null  bool   
 3   reviewTime       376892 non-null  object 
 4   reviewerID       376892 non-null  object 
 5   asin             376892 non-null  object 
 6   style            183606 non-null  object 
 7   reviewerName     376848 non-null  object 
 8   reviewText       376816 non-null  object 
 9   summary          376854 non-null  object 
 10  unixReviewTime   376892 non-null  int64  
 11  image            6990 non-null    object 
 12  category         376892 non-null  object 
 13  tech1            376892 non-null  object 
 14  description      376892 non-null  object 
 15  fit              376892 non-null  object 
 16  title            376892 non-null  o

In [187]:
### Tv Brands = Samsung, Vizio, LG, Sony, Panasonic, TCL, RCA, Toshiba, Sharp, Hisense(need to weed out non tv products)
only_tv_df = tv_df[(tv_df['brand']== 'Samsung') |
                    (tv_df['brand']== 'VIZIO') |
                  (tv_df['brand']== 'LG') |
                  (tv_df['brand']== 'Sony') |
                  (tv_df['brand']== 'Panasonic') |
                  (tv_df['brand']== 'TCL') |
                  (tv_df['brand']== 'RCA') |
                  (tv_df['brand']== 'Toshiba') |
                  (tv_df['brand']== 'Sharp') |
                  (tv_df['brand']=='Hisense')]

In [188]:
### drop unnessary columns : (style, unixReviewTime, image, tech1, fit, also_buy, category, description, feature, rank,
###                          tech2, also_view, similar_item, imageURL, imageURLHighRes, details)
only_tv_df = only_tv_df.drop(columns=['style', 'unixReviewTime', 'image', 'tech1', 'fit', 'also_buy', 
                         'tech2', 'also_view', 'similar_item', 'imageURL', 'imageURLHighRes', 'details',
                                     'category','description','feature', 'rank'])

In [189]:
### vote column has a lot of NA(Reviews that did not receive a helpful vote has no votes)
### Fill NA with 0
only_tv_df['vote'].fillna(value=0, inplace = True)

In [190]:
### drop NA values in reviewerName, reviewText, summary
only_tv_df.dropna(inplace=True)

In [194]:
### drop duplicates
only_tv_df.drop_duplicates(inplace=True)

In [191]:
### Rating : Good(overall rating >= 4) - labeled as 1, Bad(overall rating < 4) - Labeled as 0
only_tv_df['rating'] = np.where(only_tv_df['overall'] >= 4, 1, 0)

In [196]:
only_tv_df['rating'].value_counts()

1    53324
0    12778
Name: rating, dtype: int64

In [197]:
only_tv_df['overall'].value_counts()

5.0    40866
4.0    12458
1.0     4953
3.0     4749
2.0     3076
Name: overall, dtype: int64

In [198]:
only_tv_df.to_csv(r'C:\Users\brian\NLP\data\cell_phone_prep_data.csv')

## Basic Info Prior to NLP
Reviews : 66,102  
unique products : 1463  
unique reviewers : 56,356  
average rating : 4.2285  
good review : 53,324  
bad review : 12,778  
5 star : 40,866  
4 star : 12,458  
3 star : 4,749  
2 star : 3,076  
1 star : 4,953  
review with helpful votes : 21,801  
verified reviews : 51,483  
verified reviews with helpful votes : 13,549  
not verified reviews with helpful votes : 8,252

In [155]:
### import spacy to start nlp process
import spacy
nlp = spacy.load('en_core_web_sm')

In [159]:
### tokening Reviews with Spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [370]:
# list of punctuation
punctuations = string.punctuation

# list of stop words / add additional stop words
nlp = spacy.load('en_core_web_sm')
nlp.Defaults.stop_words |= {"...", "....", "tv"}
stop_words = nlp.Defaults.stop_words

# load English tokenizer
parser = English()

In [227]:
# create a tokenizer function
def spacy_tokenizer(sentence):
    # create token object
    # mytokens = parser(sentence)
    mytokens = nlp(sentence)
    # lemmatizing each token and convert to lowercase
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    # removing stop words and punctuation
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]
    
    return mytokens

In [163]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [230]:
### apply spacy tokenizer to each review
only_tv_df['token'] = only_tv_df['reviewText'].apply(spacy_tokenizer)

In [510]:
### Parts of speech by sentiment
only_tv_df['spacy_doc'] = list(nlp.pipe(only_tv_df['reviewText']))
positive_reviews = only_tv_df[only_tv_df['rating']==1]
negative_reviews = only_tv_df[only_tv_df['rating']==0]

pos_adj = [token.text.lower() for doc in positive_reviews.spacy_doc for token in doc if token.pos_=='ADJ']
neg_adj = [token.text.lower() for doc in negative_reviews.spacy_doc for token in doc if token.pos_=='ADJ']
adj = [token.text.lower() for doc in only_tv_df.spacy_doc for token in doc if token.pos_=='ADJ']

pos_noun = [token.text.lower() for doc in positive_reviews.spacy_doc for token in doc if token.pos_=='NOUN']
neg_noun = [token.text.lower() for doc in negative_reviews.spacy_doc for token in doc if token.pos_=='NOUN']
noun = [token.text.lower() for doc in only_tv_df.spacy_doc for token in doc if token.pos_=='NOUN']

In [241]:
from collections import Counter

In [511]:
Counter(adj).most_common(15)

[('great', 38175),
 ('good', 30537),
 ('other', 18361),
 ('better', 13141),
 ('more', 11858),
 ('easy', 10639),
 ('best', 10081),
 ('nice', 9757),
 ('new', 9694),
 ('little', 8579),
 ('same', 8333),
 ('old', 8233),
 ('few', 7993),
 ('excellent', 7266),
 ('many', 7114)]

In [512]:
Counter(noun).most_common(15)

[('tv', 145610),
 ('picture', 49229),
 ('quality', 24606),
 ('screen', 20495),
 ('sound', 19656),
 ('price', 18990),
 ('set', 16966),
 ('time', 14662),
 ('cable', 12944),
 ('remote', 12748),
 ('box', 11597),
 ('room', 10544),
 ('settings', 9919),
 ('problem', 9356),
 ('video', 8389)]

In [242]:
Counter(pos_adj).most_common(10)

[('great', 34129),
 ('good', 24947),
 ('other', 14163),
 ('better', 10706),
 ('easy', 9706),
 ('more', 9351),
 ('best', 8981),
 ('nice', 8250),
 ('little', 7181),
 ('new', 7173)]

In [243]:
Counter(neg_adj).most_common(10)

[('good', 5590),
 ('other', 4198),
 ('great', 4046),
 ('same', 2698),
 ('new', 2521),
 ('more', 2507),
 ('better', 2435),
 ('bad', 1972),
 ('few', 1911),
 ('old', 1858)]

In [244]:
Counter(pos_noun).most_common(10)

[('tv', 113482),
 ('picture', 41156),
 ('quality', 20497),
 ('price', 16702),
 ('sound', 16276),
 ('screen', 15294),
 ('set', 13811),
 ('cable', 10805),
 ('time', 10693),
 ('remote', 10079)]

In [245]:
Counter(neg_noun).most_common(10)

[('tv', 32128),
 ('picture', 8073),
 ('screen', 5201),
 ('quality', 4109),
 ('time', 3969),
 ('problem', 3684),
 ('sound', 3380),
 ('set', 3155),
 ('remote', 2669),
 ('issue', 2465)]

In [335]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2), stop_words=stop_words, token_pattern="\\b[a-z][a-z]+\\b")
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer, stop_words=stop_words)

### NMF

In [307]:
##Hisense TV only
Hisense_df = only_tv_df[only_tv_df['brand'] == 'Hisense']
Hisense_df = Hisense_df.reset_index()
corpus = Hisense_df['reviewText']
Hisense_review_word_matrix = tfidf_vector.fit_transform(corpus)
vocab = tfidf_vector.get_feature_names()

C:\Users\brian\.conda\envs\metis\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['d', 'far', 'll', 'm', 'regard', 'use', 've', '’'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [308]:
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
nmf = NMF(n_components = 5)
nmf.fit(Hisense_review_word_matrix)

C:\Users\brian\.conda\envs\metis\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


NMF(n_components=5)

In [309]:
review_topic_matrix = nmf.transform(Hisense_review_word_matrix)

review_topic_matrix_df = pd.DataFrame(review_topic_matrix).add_prefix('topic_')

review_topic_matrix_df[['reviews', 'token']] = Hisense_df[['reviewText', 'token']]
review_topic_matrix_df.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,reviews,token
0,0.039715,0.000000,0.000000,0.000596,0.002245,Seldom use 3D BUT ALL ELSE FANTASTIC just worr...,"[seldom, use, 3d, fantastic, worry, buy, long,..."
1,0.095577,0.000000,0.000000,0.016886,0.000000,Very nice TV. Can't say the same thing for the...,"[nice, tv, thing, app, useful, app, netflix, v..."
2,0.142040,0.021794,0.045465,0.047404,0.000638,Wow! This TV is awesome. The picture is so cri...,"[wow, tv, awesome, picture, crisp, clear, beat..."
3,0.000000,0.198972,0.171647,0.000000,0.000101,this was a great deal on a great product i rea...,"[great, deal, great, product, love, tv, highly..."
4,0.082239,0.000000,0.000000,0.000000,0.000000,"Bought a 50"" Hisense TV in April, by December ...","[buy, 50, hisense, tv, april, december, pictur..."


In [293]:
word_topic_matrix_df = pd.DataFrame(nmf.components_, columns=vocab).T.add_prefix('topic_')
word_topic_matrix_df.head()

,topic_0,topic_1,topic_2,topic_3,topic_4
'',0.008325,0.0,0.000000,0.000000,0.0
):,0.010816,0.0,0.000669,0.000000,0.0
-(][edit,0.004618,0.0,0.000000,0.000000,0.0
--,0.027476,0.0,0.000000,0.001030,0.0
------------------------------,0.008477,0.0,0.000000,0.000611,0.0


In [286]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [310]:
display_topics(nmf, tfidf_vector.get_feature_names(), 10)


Topic  0
tv, 4k, use, hisense, like, set, sound, remote, app, picture

Topic  1
great, tv, price, picture, deal, sound, excellent, buy, bedroom, room

Topic  2
love, daughter, tv, deal, wife, ju, feature, 5, college, clear

Topic  3
good, price, picture, quality, far, sound, value, clear, purchase, nice

Topic  4
work, perfect, perfectly, great, remote, use, issue, month, expect, need


In [268]:
def top_reviews(tweet_topic_matrix_df, topic, n_tweets):
    return (tweet_topic_matrix_df
            .sort_values(by=topic, ascending=False)
            .head(n_tweets)['reviews']
            .values)

def top_words(word_topic_matrix_df, topic, n_words):
    return (word_topic_matrix_df
            .sort_values(by=topic, ascending=False)
            .head(n_words))[topic]

In [294]:
for review in top_reviews(review_topic_matrix_df, 'topic_0', 15):
    print(review)

Note: Updates at the end. Original review:

I was surprised at how good this HiSense 4K HD TV turned out. At a modest price for 4K resolution I didn't expect to be impressed by quality, design or features. I was wrong. I think this is a excellent TV value at this price and has given me a very good picture. I've lived with this TV for about 6 weeks.

Important features for me are high quality upscaling, a good ATSC tuner, Apps, media access (played from Plex and windows media on my network), USB ports (to power my antenna amplifier and for media), and a responsive interface. The most important apps of the many included are Amazon Instant Streaming, Netflix, and Youtube.  All features and apps are at least as good as I expected for an econcomy TV and often exceeded my expectations.

NOTE: This TV does NOT include 3D support. Using 3D always gives me a headache, so this is not a negative for me at all.

Picture quality has been better than I expected. In my experience watching NFL Footbal

In [320]:
Hisense_df = only_tv_df[only_tv_df['brand'] == 'Hisense']
Hisense_df = Hisense_df.reset_index()
pos_Hisense_df = Hisense_df[Hisense_df['rating'] == 1].reset_index()

In [321]:
pos_corpus = pos_Hisense_df['reviewText']
pos_Hisense_review_word_matrix = tfidf_vector.fit_transform(pos_corpus)

In [322]:
nmf = NMF(n_components = 5)
nmf.fit(pos_Hisense_review_word_matrix)

C:\Users\brian\.conda\envs\metis\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


NMF(n_components=5)

In [324]:
vocab = tfidf_vector.get_feature_names()
review_topic_matrix = nmf.transform(pos_Hisense_review_word_matrix)
review_topic_matrix_df = pd.DataFrame(review_topic_matrix).add_prefix('topic_')
review_topic_matrix_df[['reviews', 'token']] = pos_Hisense_df[['reviewText', 'token']]
word_topic_matrix_df = pd.DataFrame(nmf.components_, columns=vocab).T.add_prefix('topic_')
display_topics(nmf, tfidf_vector.get_feature_names(), 10)


Topic  0
tv, 4k, use, hisense, like, set, sound, remote, picture, app

Topic  1
great, tv, price, picture, deal, excellent, buy, sound, bedroom, guest

Topic  2
love, daughter, tv, deal, wife, ju, college, feature, 5, new

Topic  3
good, price, picture, quality, far, sound, value, clear, purchase, cheap

Topic  4
work, perfect, perfectly, great, use, remote, expect, issue, month, need


In [333]:
neg_Hisense_df = Hisense_df[Hisense_df['rating'] == 0].reset_index()
neg_corpus = neg_Hisense_df['reviewText']
neg_Hisense_review_word_matrix = tfidf_vector.fit_transform(neg_corpus)
nmf = NMF(n_components = 5)
nmf.fit(neg_Hisense_review_word_matrix)
vocab = tfidf_vector.get_feature_names()
review_topic_matrix = nmf.transform(neg_Hisense_review_word_matrix)
review_topic_matrix_df = pd.DataFrame(review_topic_matrix).add_prefix('topic_')
review_topic_matrix_df[['reviews', 'token']] = neg_Hisense_df[['reviewText', 'token']]
display_topics(nmf, tfidf_vector.get_feature_names(), 10)

C:\Users\brian\.conda\envs\metis\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
tv, screen, look, 4k, motion, like, video, setting, watch, set

Topic  1
quality, poor, tv, picture, return, line, vertical, fire, image, pleased

Topic  2
sound, hisense, month, tv, 3, fix, 4, time, channel, hook

Topic  3
tv, connect, roku, network, wifi, buy, wireless, use, internet, work

Topic  4
refund, stop, week, amazon, work, break, s, internally, crack, pay


C:\Users\brian\.conda\envs\metis\lib\site-packages\sklearn\decomposition\_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


## LDA
**Hisense Negative**

In [336]:
from gensim import corpora, models, similarities, matutils
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [455]:
# create bag of words vectorizer object
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words=stop_words, token_pattern="\\b[a-z][a-z]+\\b")

In [456]:
# fit the vocab in the vectorizer
count_vectorizer.fit(neg_corpus)

C:\Users\brian\.conda\envs\metis\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CountVectorizer(ngram_range=(1, 2),
                stop_words={"'d", "'ll", "'m", "'re", "'s", "'ve", '...',
                            '....', 'a', 'about', 'above', 'across', 'after',
                            'afterwards', 'again', 'against', 'all', 'almost',
                            'alone', 'along', 'already', 'also', 'although',
                            'always', 'am', 'among', 'amongst', 'amount', 'an',
                            'and', ...},
                token_pattern='\\b[a-z][a-z]+\\b')

In [457]:
# transform the data into format
neg_Hisense_review_word = count_vectorizer.transform(neg_corpus).transpose()

In [458]:
pd.DataFrame(neg_Hisense_review_word.toarray(), count_vectorizer.get_feature_names()).head()

,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ability straight,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ability upscale,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able avoid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [459]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(neg_Hisense_review_word)
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [460]:
# create LDA model
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=10)

In [461]:
dictionary = Dictionary([list(count_vectorizer.vocabulary_.keys())])
vis = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis)

**Hisense cons**  
topic1 - picture, screen quality  
topic2 - network, wifi connecting problems  
topic3 - wireless connection  
topic6 - motion interpolation

In [380]:
lda.print_topics()

[(0,
  '0.006*"picture" + 0.006*"good" + 0.005*"sound" + 0.004*"screen" + 0.004*"like" + 0.003*"time" + 0.003*"work" + 0.003*"hisense" + 0.003*"network" + 0.003*"settings"'),
 (1,
  '0.006*"mail" + 0.004*"samsung" + 0.004*"registration" + 0.004*"address" + 0.003*"native" + 0.003*"customer" + 0.003*"said" + 0.003*"support" + 0.003*"mail address" + 0.003*"picture"'),
 (2,
  '0.003*"use" + 0.003*"image" + 0.003*"monitor" + 0.002*"quality" + 0.002*"returned" + 0.002*"poor" + 0.002*"inches" + 0.002*"got" + 0.001*"hisense" + 0.001*"picture"'),
 (3,
  '0.004*"picture" + 0.004*"sync" + 0.004*"lip" + 0.004*"lip sync" + 0.003*"like" + 0.003*"hisense" + 0.003*"problem" + 0.003*"audio" + 0.003*"sound" + 0.003*"customer"'),
 (4,
  '0.005*"content" + 0.005*"skin" + 0.004*"product" + 0.004*"skin softening" + 0.004*"softening" + 0.003*"quality" + 0.003*"sound" + 0.003*"hd" + 0.003*"makes" + 0.003*"years"'),
 (5,
  '0.007*"hisense" + 0.006*"eula" + 0.005*"motion" + 0.004*"price" + 0.004*"amazon" + 0.00

**Hisense Positive**

In [372]:
count_vectorizer.fit(pos_corpus)
pos_Hisense_review_word = count_vectorizer.transform(pos_corpus).transpose()
pd.DataFrame(pos_Hisense_review_word.toarray(), count_vectorizer.get_feature_names()).head()
corpus = matutils.Sparse2Corpus(pos_Hisense_review_word)
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=10)
lda.print_topics()

C:\Users\brian\.conda\envs\metis\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[(0,
  '0.004*"great" + 0.003*"picture" + 0.003*"works" + 0.003*"price" + 0.002*"screen" + 0.002*"little" + 0.002*"hisense" + 0.002*"time" + 0.002*"sound" + 0.002*"use"'),
 (1,
  '0.006*"great" + 0.005*"picture" + 0.003*"sound" + 0.003*"amazon" + 0.003*"quality" + 0.003*"good" + 0.003*"hisense" + 0.003*"deal" + 0.002*"netflix" + 0.002*"works"'),
 (2,
  '0.007*"picture" + 0.006*"good" + 0.005*"hisense" + 0.005*"quality" + 0.004*"great" + 0.004*"settings" + 0.003*"price" + 0.003*"apps" + 0.003*"sound" + 0.002*"like"'),
 (3,
  '0.013*"great" + 0.008*"picture" + 0.006*"price" + 0.005*"good" + 0.004*"sound" + 0.003*"hisense" + 0.003*"smart" + 0.003*"hdr" + 0.003*"amazon" + 0.003*"easy"'),
 (4,
  '0.008*"great" + 0.006*"price" + 0.005*"good" + 0.003*"picture" + 0.002*"sound" + 0.002*"quality" + 0.002*"netflix" + 0.002*"amazon" + 0.002*"tvs" + 0.002*"hisense"'),
 (5,
  '0.005*"picture" + 0.004*"set" + 0.004*"good" + 0.004*"price" + 0.003*"hisense" + 0.003*"xbox" + 0.003*"great" + 0.003*"sound

In [504]:
### class to automate extration of each company's dataframe
class company:
    
    def __init__(self, name):
        self.name = name
    
    def dataframe(self, sentiment = 'all'):
        if sentiment == 'pos':
            return only_tv_df[(only_tv_df['brand'] == self.name) & (only_tv_df['rating'] == 1)].reset_index()
        elif sentiment == 'neg':
            return only_tv_df[(only_tv_df['brand'] == self.name) & (only_tv_df['rating'] == 0)].reset_index()
        elif sentiment == 'all':
            return only_tv_df[(only_tv_df['brand'] == self.name)].reset_index()

**Samsung Negative**

In [516]:
samsung = company('Samsung')
df1 = samsung.dataframe('neg')

In [517]:
corpus = df1['reviewText']
count_vectorizer.fit(corpus)
review_word =  count_vectorizer.transform(corpus).transpose()
corpus = matutils.Sparse2Corpus(review_word)
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=10)
lda.print_topics()

[(0,
  '0.001*"app" + 0.001*"work" + 0.000*"smart" + 0.000*"smarthub" + 0.000*"apps crash" + 0.000*"slow" + 0.000*"response time" + 0.000*"apps" + 0.000*"youtube app" + 0.000*"slow slow"'),
 (1,
  '0.000*"subtitles" + 0.000*"pixel" + 0.000*"similar" + 0.000*"remote" + 0.000*"wear" + 0.000*"burned" + 0.000*"smart" + 0.000*"cnn" + 0.000*"dlna" + 0.000*"covered"'),
 (2,
  '0.016*"samsung" + 0.008*"picture" + 0.005*"like" + 0.005*"screen" + 0.005*"good" + 0.004*"set" + 0.004*"remote" + 0.004*"quality" + 0.004*"sound" + 0.004*"smart"'),
 (3,
  '0.001*"repair" + 0.001*"samsung" + 0.001*"xaa" + 0.001*"years" + 0.001*"power supply" + 0.001*"xaa xaa" + 0.001*"xfinity" + 0.001*"supply" + 0.000*"adapter" + 0.000*"power"'),
 (4,
  '0.001*"images" + 0.001*"picture" + 0.001*"class" + 0.001*"spacing" + 0.001*"video block" + 0.001*"div" + 0.001*"images amazon" + 0.001*"class video" + 0.001*"hidden value" + 0.001*"type hidden"'),
 (5,
  '0.010*"samsung" + 0.003*"smart" + 0.002*"buy" + 0.002*"service" +

In [441]:
### using pyLDAvis as visual tool
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [454]:
from gensim.corpora import Dictionary
dictionary = Dictionary([list(count_vectorizer.vocabulary_.keys())])

In [452]:
vis = gensimvis.prepare(lda, corpus, dictionary)

In [453]:
pyLDAvis.display(vis)

**Samsung cons**  
topic1 - dark scenes blur  
topic2 - customer support/warranty  
topic3 - roku app(other various apps)  

**Samsung All**

In [508]:
samsung = company('Samsung')
all_samsung_df = samsung.dataframe()

In [514]:
corpus = all_samsung_df['reviewText']
count_vectorizer.fit(corpus)
review_word =  count_vectorizer.transform(corpus).transpose()
corpus = matutils.Sparse2Corpus(review_word)
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=10)

In [515]:
dictionary = Dictionary([list(count_vectorizer.vocabulary_.keys())])
vis = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis)

MemoryError: Unable to allocate 47.8 MiB for an array with shape (626416, 10) and data type float64

**LG Negative**

In [518]:
lg = company('LG')
df1 = lg.dataframe('neg')

In [519]:
corpus = df1['reviewText']
count_vectorizer.fit(corpus)
review_word =  count_vectorizer.transform(corpus).transpose()
corpus = matutils.Sparse2Corpus(review_word)
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=10)

In [477]:
dictionary = Dictionary([list(count_vectorizer.vocabulary_.keys())])
vis = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis)

**LG cons**  
topic1 - native resolution, aspect ratio  
topic2 - active HDR problems(auto adjusting)  
topic3 - refresh rate -> edge bleeding, black instablility

## Clustering

In [480]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score

In [478]:
no_of_clusters = 10
model = KMeans(n_clusters=no_of_clusters, init='k-means++', max_iter=100, n_init=1)
model.fit(review_word)

KMeans(max_iter=100, n_clusters=10, n_init=1)

In [520]:
only_tv_df.to_csv(r'C:\Users\brian\NLP\data\cell_phone_prep_data.csv')